# Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import math
import glob
import os

# Import Data

In [41]:
# Read in the data
path = 'Data/'
files = [os.path.join(path, file) for file in os.listdir(path) if file.startswith('property_transactions_') and file.endswith('.csv')]
print(len(files), "files found.")
df_trans = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
# Remove duplicates
df_trans = df_trans.drop_duplicates()
df_trans['contractYear'] = df_trans['contractDate'].apply(lambda x: x % 100 + 2000)
df_trans['contractMonth'] = df_trans['contractDate'].apply(lambda x: x // 100)
df_trans['tenure'] = df_trans['tenure'].str.lower()     # Convert to lowercase for consistency
df_trans['tenureDuration'] = df_trans['tenure'].apply(lambda x: np.inf if 'freehold' in str(x) else int(x.split(' ')[0]) if isinstance(x, str) and len(x.split(' ')) > 1 else np.nan)      # If 'freehold' then 0, if NaN then NaN
df_trans['tenureStartYear'] = df_trans['tenure'].apply(lambda x: int(x.split(' ')[-1]) if isinstance(x, str) and len(x.split(' ')) > 3 else np.nan)     # If 'freehold' or '99 years lease' then nan
df_trans['tenureRemaining'] = df_trans.apply(lambda x: np.inf if 'freehold' in str(x['tenure']) else (x['tenureDuration'] - (x['contractYear'] - x['tenureStartYear'])) if not pd.isna(x['tenureStartYear']) else np.nan, axis=1)

group_cols = ["project", "street", "district", "area", "x", "y", "typeOfArea", "propertyType", 'tenureStartYear', "marketSegment"]

df_trans = df_trans.sort_values(by=group_cols+["contractYear"]).reset_index(drop=True)
df_trans = (
    df_trans.groupby(group_cols+["contractYear"])
    .agg(
        mean_price=("price", "mean"),
        transaction_count=("price", "count")
    )
    .reset_index()
)
df_trans["mean_price"] = df_trans["mean_price"].round(2)
df_trans['price_psm'] = df_trans['mean_price'] / df_trans['area']
df_trans['price_psm'] = df_trans['price_psm'].round(2)

df_trans.head()

4 files found.


,project,street,district,area,x,y,typeOfArea,propertyType,tenureStartYear,marketSegment,contractYear,mean_price,transaction_count,price_psm
0,1 CANBERRA,CANBERRA DRIVE,27,87.0,27611.4486,46578.21809,Strata,Executive Condominium,2012.0,OCR,2020,840000.00,1,9655.17
1,1 CANBERRA,CANBERRA DRIVE,27,87.0,27611.4486,46578.21809,Strata,Executive Condominium,2012.0,OCR,2021,884962.67,3,10171.98
2,1 CANBERRA,CANBERRA DRIVE,27,87.0,27611.4486,46578.21809,Strata,Executive Condominium,2012.0,OCR,2023,1109000.00,2,12747.13
3,1 CANBERRA,CANBERRA DRIVE,27,87.0,27611.4486,46578.21809,Strata,Executive Condominium,2012.0,OCR,2025,1230000.00,1,14137.93
4,1 CANBERRA,CANBERRA DRIVE,27,88.0,27611.4486,46578.21809,Strata,Executive Condominium,2012.0,OCR,2020,873571.43,7,9926.95


In [42]:
# Read in the data
path = 'Data/'
files = [os.path.join(path, file) for file in os.listdir(path) if file.startswith('property_rentals') and file.endswith('.csv')]
print(len(files), "files found.")
df = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
df['contractYear'] = df['leaseDate'].apply(lambda x: x % 100 + 2000)
group_cols = ["project", "street", "district", "areaSqm", "x", "y", "propertyType"]
df = df.sort_values(by=group_cols+["contractYear"]).reset_index(drop=True)
df = (
    df.groupby(group_cols+["contractYear"])
    .agg(
        mean_rent=("rent", "mean"),
        rent_count=("rent", "count")
    )
    .reset_index()
)
df = df.drop(columns=['propertyType'])
df.head()

21 files found.


C:\Users\Porkc\AppData\Local\Temp\ipykernel_26860\3284140052.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)


,project,street,district,areaSqm,x,y,contractYear,mean_rent,rent_count
0,# 1 LOFT,LORONG 24 GEYLANG,14.0,120-130,33581.81022,32778.97962,2020.0,3150.0,1
1,# 1 LOFT,LORONG 24 GEYLANG,14.0,120-130,33581.81022,32778.97962,2021.0,3375.0,4
2,# 1 LOFT,LORONG 24 GEYLANG,14.0,120-130,33581.81022,32778.97962,2022.0,4000.0,1
3,# 1 LOFT,LORONG 24 GEYLANG,14.0,120-130,33581.81022,32778.97962,2023.0,4800.0,1
4,# 1 LOFT,LORONG 24 GEYLANG,14.0,120-130,33581.81022,32778.97962,2024.0,4650.0,2


In [43]:
print(len(df))
print(len(df['project'].unique()), "unique projects")
print(len(df_trans))
print(len(df_trans['project'].unique()), "unique projects")

86699
3169 unique projects
29478
874 unique projects


# Map Rental Data to Property Transaction Data

In [44]:
def parse_range(val):
    if isinstance(val, str) and "-" in val:
        a, b = val.split("-")
        return float(a), float(b)
    try:
        v = float(val)
        return v, v
    except:
        return np.nan, np.nan

df[["area_min", "area_max"]] = df["areaSqm"].apply(lambda x: pd.Series(parse_range(x)))


In [45]:
datasample = pd.merge(
    df,
    df_trans,
    on=["street", "project", "district", "contractYear"],
    how="inner",
    suffixes=("_rent", "_sale")
)

# Now filter to keep only rows where sale.area falls in the rental range
datasample = datasample[
    (datasample["area"] >= datasample["area_min"]) &
    (datasample["area"] <= datasample["area_max"])
]

datasample = datasample.drop(columns=['area_min', 'area_max', 'areaSqm', 'x_rent', 'y_rent', 'x_sale', 'y_sale'])


In [46]:
print(datasample.head())

       project          street  district  contractYear  mean_rent  rent_count  \
6   1 CANBERRA  CANBERRA DRIVE      27.0        2023.0     3650.0           4   
7   1 CANBERRA  CANBERRA DRIVE      27.0        2023.0     3650.0           4   
8   1 CANBERRA  CANBERRA DRIVE      27.0        2023.0     3650.0           4   
9   1 CANBERRA  CANBERRA DRIVE      27.0        2023.0     3650.0           4   
23  1 CANBERRA  CANBERRA DRIVE      27.0        2021.0     3250.0           1   

     area typeOfArea           propertyType  tenureStartYear marketSegment  \
6   116.0     Strata  Executive Condominium           2012.0           OCR   
7   117.0     Strata  Executive Condominium           2012.0           OCR   
8   118.0     Strata  Executive Condominium           2012.0           OCR   
9   119.0     Strata  Executive Condominium           2012.0           OCR   
23  112.0     Strata  Executive Condominium           2012.0           OCR   

    mean_price  transaction_count  price_psm

In [47]:
print(len(datasample))
print(len(datasample['project'].unique()), "unique projects")

29211
742 unique projects


# Investigate Relationship Between Rental And Property Price

In [48]:
# Clean numeric fields
datasample["mean_rent"] = pd.to_numeric(datasample["mean_rent"], errors="coerce")
datasample["mean_price"] = pd.to_numeric(datasample["mean_price"], errors="coerce")
datasample["price_psm"] = pd.to_numeric(datasample["price_psm"], errors="coerce")
datasample["area"] = pd.to_numeric(datasample["area"], errors="coerce")

datasample["gross_rental_yield"] = (datasample["mean_rent"] * 12) / datasample["mean_price"]

In [49]:
yield_summary = (
    datasample.groupby(["district", "marketSegment"])["gross_rental_yield"]
    .agg(["mean", "median", "std", "count"])
    .sort_values("mean", ascending=False)
)
print(yield_summary.head(10))


                            mean    median       std  count
district marketSegment                                     
2.0      CCR            0.039030  0.039400  0.006237    267
25.0     OCR            0.038143  0.039103  0.006697    986
2.0      RCR            0.037970  0.037000  0.005776     60
14.0     OCR            0.037075  0.036179  0.006834    409
22.0     OCR            0.036742  0.036562  0.005591   1166
27.0     OCR            0.036571  0.036459  0.005453   1876
14.0     RCR            0.036119  0.035487  0.005894    564
5.0      RCR            0.035967  0.035278  0.004823    370
1.0      CCR            0.035947  0.036449  0.006838    477
18.0     OCR            0.035588  0.035381  0.005865   2808


In [52]:
yield_summary = (
    datasample.groupby(["propertyType", "marketSegment"])["gross_rental_yield"]
    .agg(["mean", "median", "std", "count"])
    .sort_values("mean", ascending=False)
)
print(yield_summary.head(10))

                                         mean    median       std  count
propertyType          marketSegment                                     
Strata Detached       RCR            0.054779  0.055385  0.003743      3
Detached              RCR            0.040925  0.023290  0.049737     15
Executive Condominium OCR            0.036039  0.036195  0.006139   4262
Strata Semi-detached  RCR            0.035786  0.035786  0.006970      2
Apartment             CCR            0.035411  0.035172  0.006870   1305
                      RCR            0.035278  0.034837  0.006415   1830
                      OCR            0.034882  0.034303  0.006242   2768
Condominium           OCR            0.034184  0.033797  0.006126  11355
Strata Semi-detached  CCR            0.033470  0.034349  0.003091     24
Strata Terrace        CCR            0.033057  0.032532  0.003237    228


In [53]:

yield_trend = (
    datasample.groupby("contractYear")["gross_rental_yield"]
    .mean()
    .reset_index()
)

In [55]:
growth = (
    datasample.groupby("contractYear")[["mean_rent", "mean_price"]]
    .mean()
    .pct_change()
    .rename(columns={"mean_rent": "rent_growth", "mean_price": "price_growth"})
)

print(growth)

              rent_growth  price_growth
contractYear                           
2020.0                NaN           NaN
2021.0           0.175081      0.208615
2022.0           0.178851      0.045841
2023.0           0.144090      0.026116
2024.0          -0.049820      0.027872
2025.0           0.019894      0.025435


In [56]:
def classify_market(row):
    if row["price_growth"] < 0 and row["rent_growth"] > 0:
        return "Buyer's & Landlord's Market"
    elif row["price_growth"] > 0 and row["rent_growth"] < 0:
        return "Seller's Market"
    elif row["price_growth"] > 0 and row["rent_growth"] > 0:
        return "Hot Market (Seller's)"
    elif row["price_growth"] < 0 and row["rent_growth"] < 0:
        return "Cooling / Weak Market"
    else:
        return "Stable / Balanced"

growth["market_type"] = growth.apply(classify_market, axis=1)
print(growth)

              rent_growth  price_growth            market_type
contractYear                                                  
2020.0                NaN           NaN      Stable / Balanced
2021.0           0.175081      0.208615  Hot Market (Seller's)
2022.0           0.178851      0.045841  Hot Market (Seller's)
2023.0           0.144090      0.026116  Hot Market (Seller's)
2024.0          -0.049820      0.027872        Seller's Market
2025.0           0.019894      0.025435  Hot Market (Seller's)
